#### Second attempt at the python version of the centralised part of the microarray methylation analysis workflow (Quality control upto normalisation)
Using python as a shell to string together the specialised r functions used in the Exeter workflow

Loading in the required modules/packages

In [1]:
import pandas as pd
import numpy as np
import subprocess
import csv
import glob
import os
import re
import seaborn as sns
from matplotlib import pyplot as plt

# stuff needed for some specific analysis - maybe not needed in this version of the code
#from sklearn.decomposition import PCA 
#from scipy.stats import pearsonr
#from sklearn.cluster import KMeans

In [2]:
working_path = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis"
data_path = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data"
output_path = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\Practical work\\Federated_Differential_Methylation_Analysis\\Output"


Use subprocess to read the data contained in the idat files into dataframe using the readEPIC function from the wateRmelon package in R

### Creating an output file structure and loading in the idat files

The input arguments of this script are: 
1. file_path to the folder containing the .idat files 
2. file_path to the phenotype information sheet (.txt) 
3. the directory where the output should be saved 
4. OPTIONAL the data identifier to be used in the creation of the output folders - this still needs to be fixed

In [7]:
load_with_option = subprocess.run(["C:\\Program Files\\R\\R-4.1.2\\bin\\Rscript.exe", '--vanilla', "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Loading_idats_code_saveOutput_python_shell.R", "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\idat", "E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_RAW\GSE66351_pheno.txt", "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\Practical work\\Federated_Differential_Methylation_Analysis\\Output", "GSE66351a"], capture_output=True)

In [ ]:
print(load_with_option.stderr)

Using subprocess to perform the complete preprocessing workflow upto the normalisation  
This is the whole preprocessing run as one function in the r-script. The script takes 5 input arguments:  
1. The file path of the folder containing the .idat files
2. The phenotype information file
3. The working directory where the output folder should be created
4. The filepath to the illumina manifest file that contains the column "CHR" with the chromosome each probe is located on
5. The identifier that should be included in the name of the output folder


In [5]:
central_preprocessing = subprocess.run(["C:\\Program Files\\R\\R-4.1.2\\bin\\Rscript.exe", '--vanilla', "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\preprocessing_r_code_replication_shell_version_no_norm_edit.r", "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\idat", "E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_RAW\GSE66351_pheno.txt", "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\Practical work\\Federated_Differential_Methylation_Analysis\\Output", "E:\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\GSE66351\\GPL13534_HumanMethylation450_15017482_v.1.1.csv", "GSE66351"], capture_output = True)

In [6]:
#check what happend in the subprocess
print(central_preprocessing.stderr)
print(central_preprocessing.stdout)

b'Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.2 (2021-11-01)\r\nInstallation paths not writeable, unable to update packages\r\n  path: C:/Program Files/R/R-4.1.2/library\r\n  packages:\r\n    class, cluster, foreign, MASS, Matrix, mgcv, nlme, nnet, rpart, spatial,\r\n    survival\r\nOld packages: \'AnnotationHub\', \'beanplot\', \'BiocManager\', \'blob\', \'broom\',\r\n  \'checkmate\', \'cli\', \'clipr\', \'colorspace\', \'commonmark\', \'crayon\', \'dplyr\',\r\n  \'DT\', \'ensembldb\', \'evaluate\', \'fansi\', \'formatR\', \'GenomicFeatures\', \'glue\',\r\n  \'Gviz\', \'haven\', \'Hmisc\', \'jsonlite\', \'knitr\', \'limma\', \'locfit\', \'magrittr\',\r\n  \'matrixStats\', \'openssl\', \'plyr\', \'processx\', \'ps\', \'RColorBrewer\', \'Rcpp\',\r\n  \'RCurl\', \'readxl\', \'reshape\', \'rhdf5\', \'rlang\', \'rmarkdown\', \'RSQLite\',\r\n  \'S4Vectors\', \'sass\', \'scales\', \'tidyselect\', \'tinytex\', \'tzdb\', \'uuid\',\r\n  \'vctrs\', \'withr\', \'xfun\', \'XML\', \'yaml

Next step is to normalise the data, this step will be offered centrally and distributed/federated to be flexible to the researchers needs  
Below a implementation of the normalisation algorithm behind the dasen function in the wateRmelon package is provided

Dasen normalisation is a form of quantile normalisation that is performed for the two probe types seperately. The normalised data (betas), per probe type, are calculated using the normalised methylated and unmethylated intensities of each probe type.  
    betas (per probe) = quantile normalised methylated intensities / (quantile normalised methylated intensities + quantile normalised unmethylated intensities + 100)  
The first step is to write the quantile normalisation function

In [19]:
# quantile normalisation function
def quantile_normalise(input_data):
    """
    input_data = a dataframe that needs to be quantile normalised
    returns a quantile normalised version of the input_data
    """
    data_sorted = pd.DataFrame(np.sort(input_data.values, axis = 0), index = input_data.index, columns = input_data.columns) #sort the values of each column (sample) and keep the original row 
    # and column names
    data_sorted_means = data_sorted.mean(axis = 1) # calulate the row means of the sorted data -> these means will be used to replace the raw values in the data
    data_sorted_means.index = np.arange(1, len(data_sorted_means)+1) # this sets the index so it will correspond to the descending ranks that will be assigned to the original 
    # data in the dataframe. This way the row means, which are sorted loweste to highest, can be used to replace the raw data in the correct order
    data_rank = input_data.rank(method = "min").stack().astype(int) # get the rank of the values for each sample in the raw dataset in integer format and change the dataframe so that
    # the columns become the rows, with a multi-index indicating probe as the highest level and the samples for that probe as the second level
    QN_data = data_rank.map(data_sorted_means).unstack() # map the row mean values onto the matching ranks obtained from the original dataframe and bring it back to a row = probe
    # and column = sample format
    return (QN_data)
# works as it should
    

Before the dasen function can be coded, first a couple of supporting functions need to be translated from r to python, these have been defined in the wateRmelon package as:  
* dfs2
* dfsfit

In [23]:
# load in some data and create a test data object to write these functions
test_methylated = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Output\\QC_GSE66351_PythonShell\\Raw_methylated_intensities.csv", index_col=0)
test_methylated = test_methylated.iloc[0:21, 0:21]

test_unmethylated = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Output\\QC_GSE66351_PythonShell\\Raw_unmethylated_intensities.csv", index_col=0)
test_unmethylated = test_unmethylated.iloc[0:21, 0:21]


In [5]:
# attach the probe type information to the (test) data so it can be used by the normalisation functions
annotation_data = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\GPL13534_HumanMethylation450_15017482_v.1.1.csv", skiprows=7, low_memory=False)
annotation_data.set_index(annotation_data["IlmnID"], inplace=True)
probe_type_data = annotation_data.loc[:, "Infinium_Design_Type"]
test_probe_annotation = pd.merge(test_methylated, probe_type_data, how = "inner", left_index=True, right_index=True, indicator = True)
test_probe_annotation = test_probe_annotation.loc[:,"Infinium_Design_Type"]

C:\Users\Silke\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (2,4,11,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
#save everything to test the next function in r
test_methylated.to_csv(os.path.join(output_path, "test_normalisation\\methylation.csv"))
test_unmethylated.to_csv(os.path.join(output_path, "test_normalisation\\unmethylation.csv"))
test_probe_annotation.to_csv(os.path.join(output_path, "test_normalisation\\probe_annotation.csv"))

In [40]:
def dfs2_python_sklear(x, probe_type = test_probe_annotation):
    from sklearn.neighbors import KernelDensity
    #KD_one = KernelDensity(kernel = "gaussian", ).fit(x[probe_type == "I"])
    #one = KD_one.score_samples(x[probe_type == "I"])
    #KD_two = KernelDensity(kernel = "gaussian", ).fit(x[probe_type == "II"])
    #two = KD_two.score_samples(x[probe_type == "II"])
    #out = np.max(one) - np.max(two) #not quite sure if any of this is correct

    # new code version that should work on one column at a time
    x_copy = x.copy()
    KD_one = KernelDensity(kernel = "gaussian", ).fit(np.array(x_copy[probe_type == "I"]).reshape(1,-1))
    one = KD_one.score_samples(np.array(x_copy[probe_type == "I"]).reshape(1,-1))
    KD_two = KernelDensity(kernel = "gaussian", ).fit(np.array(x_copy[probe_type == "II"]).reshape(1,-1))
    two = KD_two.score_samples(np.array(x_copy[probe_type == "II"]).reshape(1,-1))
    out = np.max(one) - np.max(two) #not quite sure if any of this is correct
    return out

# the code works - need to try a different desity estimator function because this returns the same value for each column

In [6]:
def dfs2_python(x, probe_type):
    import statsmodels.api as sm
    from statsmodels.distributions.mixture_rvs import mixture_rvs

    # new code version that should work on one column at a time
    x_copy = x.copy()
    KD_one = sm.nonparametric.KDEUnivariate(x_copy[probe_type == "I"])
    KD_one.fit(gridsize=5000)
    one = int(KD_one.support[np.where(np.max(KD_one.density))])
    KD_two = sm.nonparametric.KDEUnivariate(x_copy[probe_type == "II"])
    KD_two.fit(gridsize=5000)
    two = int(KD_two.support[np.where(np.max(KD_two.density))])
    out = np.max(one) - np.max(two) #not quite sure if any of this is correct
    return out

#this one works more similar to the r original although the output is about a factor 10^-5 off compared to r

In [26]:
def dfsfit_python(x, probe_type):
    import statsmodels.api as sm
    import re
    dis_diff = x.apply(dfs2_python, args = (probe_type,), axis=0) #create a dataframe/array of the values when dfs2 is applied to each column
    
    roco = []
    for col_name in test_unmethylated.columns.values.tolist() :
        found = re.search("(R0[1-9]C0[1-9])", col_name).group(1)
        roco.append(found) 
    
    srow = []
    scol = []
    for ro in roco:
        row = int(ro[2])
        srow.append(row)
        col = int(ro[5])
        scol.append(col)
    
    fit_dist = sm.OLS.from_formula("dis_diff ~ scol + srow", dis_diff).fit()
    dis_diff = [fit_dist.fittedvalues]

    tI_correction = np.tile(np.array(dis_diff), (3,1))
    x[probe_type == "I"] = x[probe_type == "I"] - tI_correction
    return x

# this works too - although the output is about a factor 10^-5 off compared to r
    

In [27]:
# dasen normalisation
def dasen_normalisation(unmethylated, methylated, probe_type, base = 100):
    """
    computes the dasen normalised beta values: quantile normalises the unmethylated and methylated intensities, per probe type,
    and uses these normalised intensities to calculate the beta values

    Input arguments:
    unmethylated = dataframe of unmethylated intensities
    methylated = dataframe of methylated intensities
    probe_type = series indicating the type of each probe (Type I or Type II)

    Returns: a dataframe of normalised beta values
    """
    # fit the probability ditribution to the methylated and unmethylated probe intensities based on their probe type
    unmethylated_fit = dfsfit_python(unmethylated, probe_type)
    methylated_fit = dfsfit_python(methylated, probe_type)

    # calculate the quantile normalised values for the methylated and unmethylated probe intensities based on the estimated distribution values
    unmethylated[probe_type == "I"] = quantile_normalise(unmethylated_fit[probe_type == "I"])
    unmethylated[probe_type == "II"] = quantile_normalise(unmethylated_fit[probe_type == "II"])

    methylated[probe_type == "I"] = quantile_normalise(methylated_fit[probe_type == "I"])
    methylated[probe_type == "II"] = quantile_normalise(methylated_fit[probe_type == "II"])

    # calculate the new beta values based on the per probe normalised methylated and unmethylated probe intentisity values
    betas = methylated/(methylated + unmethylated + base) 
    return betas
# this also works as it should 

In [28]:
test_normalised_betas = dasen_normalisation(test_unmethylated, test_methylated, test_probe_annotation)

For now, to move on to writing the EWAS code, I wrote a script around the normalisation with the dasen function and the cell type decomposition in r which will be run as a subprocess. The normalisation will be implemented in python in the final version but the cell type decomposition remains r based because there is limitted need to reimplement that in a federated fashion - THIS IS NOT WORKING, INLCUDED THE NORMALISATION AND CELL TYPE DECOMPOSITION INTO THE R-SCRIPT THAT IS RUN IN THE SUBPROCESS FOR NOW

In [71]:
normalisation = "dasen_normalisation.r"
normalisation_file = os.path.join(working_path, normalisation)
data = os.path.join(output_path, "preprocessed_MethyLumiSet.RData") 
manifest_path = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\GPL13534_HumanMethylation450_15017482_v.1.1.csv"


In [72]:
r_normalisation = subprocess.run(["C:\\Program Files\\R\\R-4.1.2\\bin\\Rscript.exe", '--vanilla', normalisation_file, data, output_dir, manifest_path], capture_output = True)

In [ ]:
print(r_normalisation.stderr)
print(r_normalisation.stdout)

R script containing the RefFreeEWAS cell type decomposition which will be run in a subprocess, output saved and added to the phenotype information that will be used in the EWAS furhter down in this file

In [18]:
# specifying the paths that go into the subprocess function
file_path = os.path.join(working_path, "RefFreeEWAS_local.r")
data_path = os.path.join(output_path, "QC_GSE66351_PythonShell", "Preprocessed_Normalised_MethyLumiSet.RData") #try again with different input data
manifest_path = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\GPL13534_HumanMethylation450_15017482_v.1.1.csv"
pheno_path = os.path.join(output_path, "QC_GSE66351_PythonShell", "post_norm_pheno_information.csv")

# RefFreeEWAS subprocess
RefFreeEWAS = subprocess.run(["C:\\Program Files\\R\\R-4.1.2\\bin\\Rscript.exe", '--vanilla', file_path, data_path, pheno_path, manifest_path], capture_output=True)

In [ ]:
RefFreeEWAS.stderr

In [ ]:
import dasen_normalisation

EWAS code, based on the least squares linear algebra as used in the fortran code at the foundation of the lm() function in r

In [10]:
#EWAS

import numpy as np
import pandas as pd

pheno = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Output\\QC_GSE66351_PythonShell\\post_norm_pheno_information.csv", index_col= "Sample_ID")
betas = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\Output\\QC_GSE66351_PythonShell\\Preprocessed_betas.csv", index_col=0)
x = pheno.loc[:,["Sample_diagnosis", "Sample_age", "Sample_sex", "Sample_sentrix_id"]] # design matrix with the dependent/explainatory variables to be included in the model
y = betas.iloc[0:21,:] # keeping it small now to test if everything works the way it should

# The design matrix needs to consist of numeric representations of the covariates to be included in the model, i.e. binary diagnosis, binary sex, dummy sentrix etc.
x["Sample_diagnosis"] = (x["Sample_diagnosis"] == "diagnosis: AD").astype(int) #create binary diagnosis with 1 = AD and 0 = CTR
x["Sample_sex"] = (x["Sample_sex"] == "Sex: F").astype(int) #create binary sex with 1 = F and 0 = M
# create dummy variables for the unique sentrix_ids present in the dataset - this code can be reused to create center number dummies in the federated version of the code
unique_ids = x["Sample_sentrix_id"].unique()
for id in unique_ids:
    x[id] = (x["Sample_sentrix_id"] == id).astype(int)
x.drop(columns="Sample_sentrix_id", inplace = True)
# turn the age variable into a continuous numerical variable without any leftover text
x["Sample_age"].replace("^[^:]*:", "", regex=True, inplace=True)
x["Sample_age"] = pd.to_numeric(x["Sample_age"])

def EWAS_central(design_matrix, beta_values):
    x_matrix = design_matrix.values
    y_matrix = beta_values.values


    n = y_matrix.shape[0] # select the number of rows of the beta matrix - #genes that the linear model will be calculated for
    m = x.shape[1] #select the number of columns from the design matrix

    import scipy.stats

    coefficient = []
    standard_error = []
    t_stat = []
    p_value = []
    for i in range(0, n):
        y_m = y_matrix[i, :]
        x_t = x_matrix.T @ x_matrix
        x_t_y = x_matrix.T @ y_m
        x_t_inv = np.linalg.inv(x_t)
        coef = x_t_inv @ x_t_y
        coefficient.append(coef)
        stan_er = np.diag(x_t_inv)
        standard_error.append(stan_er)
        t = coef/stan_er
        t_stat.append(t)
        df = y_matrix.shape[1]-2 #degrees of freedom is defined as number of observations - 2 
        p = scipy.stats.t.sf(t, df)
        p_value.append(p)

#turn the results saved in lists into a dataframe for each covariate with the probe ids as index
    result_coef = pd.DataFrame(coefficient, index=y.index, columns=x.columns)
    result_staner = pd.DataFrame(standard_error, index = y.index, columns=x.columns)
    result_tvalue = pd.DataFrame(t_stat, index=y.index, columns=x.columns)
    result_pvalue = pd.DataFrame(p_value, index=y.index, columns=x.columns)

#create a final results dataframe that contains the coefficient, standard error and p-value of the diagnosis covariate included in the linear regression
    results_diagnosis = pd.DataFrame({"Diagnosis_Coef":result_coef["Sample_diagnosis"], "Diagnosis_StanErr":result_staner["Sample_diagnosis"], "Diagnosis_Pvalue":result_pvalue["Sample_diagnosis"]}, index=y.index)
    results_diagnosis.to_csv(os.path.join(output_path, "results_diagnosis_regression_test_python.csv"))
    complete_results = pd.concat({"Coefficient":result_coef, "StandardError":result_staner, "P-value":result_pvalue}, axis = 1)
    results_diagnosis.to_csv(os.path.join(output_path, "complete_results_regression_test_python.csv"))
    return results_diagnosis, complete_results

  


Create a .bed structured text file with the regression output to be used as input into the differentially methylated region analysis

In [63]:
# start with importing the probe information from the .bed file that is available through the encord project (?)
bed_annotation = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\HAIB.A549.EtOH.Rep.3.bed", sep="\t", header=None)
# select the three necessary column, chr, start, stop from the annotation file and match these to the probes in the EWAS input betas based on probe ID
bed_annotation = bed_annotation.iloc[:,0:4]
bed_annotation.columns = ["chr", "ChromStart", "ChromEnd", "Illumina_ID"]
# merge the regression output onto the .bed standard columns based on the probe ID
results_bed = pd.merge(bed_annotation, results_diagnosis, left_on="Illumina_ID", right_index=True, how="inner") #using inner join since this preserves the order of the keys and
# only keeps the entries that are present in both dataframes
results_bed.set_index(results_bed["Illumina_ID"], inplace=True)
# write the dataframe as a tab separated .bed file
results_bed.to_csv(os.path.join(output_path, "results_diagnosis_regression_test.bed"), sep="\t")

In [1]:
import pandas as pd 
import numpy as np